In [1]:
# part 1
import numpy as np

filename = "data.txt"
with open(filename, 'r') as f:
    s = f.read()
    initial, compuatations = s.split("\n\n")

compuatations = compuatations.split("\n")

outcomes, dependencies, operations = [], [], []
for compuatation in compuatations:
    args, outcome = compuatation.split(" -> ")
    args = args.split(" ")

    dependencies.append((args[0], args[2]))
    operations.append(args[1])
    outcomes.append(outcome)

def comp(a, b, op):
    if op == 'XOR':
        return a ^ b
    elif op == 'OR':
        return a | b
    elif op == 'AND':
        return a & b
    

to_do = list(range(len(outcomes)))
done = {x.split(": ")[0]: bool(int(x.split(": ")[1])) for x in initial.split("\n")}

while to_do:
    for i in to_do:
        a, b = dependencies[i]
        if a in done and b in done:
            done[outcomes[i]] = comp(done[a], done[b], operations[i])
            to_do.remove(i)
            break


print(sum([2 ** i * done.get('z' + ["0", ''][i >= 10] + str(i), 0) for i in range(100)]))


59619940979346


In [2]:
# part 2

def get_dependencies(name): # find every x and y (input values) that name depends on
    if name[0] in ['x', 'y']:
        return {name}

    for i in range(len(outcomes)):
        if outcomes[i] == name:
            a, b = dependencies[i] 
            return get_dependencies(a) | get_dependencies(b)
        


def get_influences_below_i(name, max_i):
    infs = set()
    for i in range(len(dependencies)):
        a, b = dependencies[i]
        c = outcomes[i]

        if name in (a, b):
            other = {a: b, b: a}[name]
            deps = get_dependencies(other) | {other}

            if not sum([dep[0] in ['x', 'y'] and int(dep[1:]) > max_i for dep in deps]): # elements do not depend on x / y greater than 1
                infs = infs | {c} | get_influences_below_i(c, max_i)
    
    return infs



In [3]:

affects = {}
for i in range(len(dependencies)):
    a, b = dependencies[i]
    c = outcomes[i]

    affects[a] = affects.get(a, []) + [c]
    affects[b] = affects.get(b, []) + [c]


def compute(local_results, pos):
    prev_l, l = -1, 0

    while l > prev_l: # did we make progress?
        prev_l = l
        if pos in local_results:
            return local_results[pos]

        for i in range(len(dependencies)):
            a, b = dependencies[i]
            if a in local_results and b in local_results:
                a, b = local_results[a], local_results[b]
                local_results[outcomes[i]] = comp(a, b, operations[i])
        
        l = len(local_results)


        
def find_carry_and_z(x, y, c, possible_cz):
    poss_start_values = [ {x: xv, y: yv, c: cv} for xv in [0, 1] for yv in [0, 1] for cv in [0, 1]] # 8 possible combinations
    poss_carry_bits, poss_outc_bits = [], []
    
    for cz in possible_cz:
        ress = [compute(poss_start_values[i].copy(), cz) for i in range(8)]
            
        if ress == [0, 0, 0, 1, 0, 1, 1, 1]: poss_carry_bits.append(cz)
        if ress == [0, 1, 1, 0, 1, 0, 0, 1]: poss_outc_bits.append(cz)

    return poss_carry_bits, poss_outc_bits



In [4]:

def swap(x, y):
    if y in affects.get(x, []) or x in affects.get(y, []):
        return # it is impossible to swap these because it would cause a loop

    x_i, y_i = None, None
    for i in range(len(dependencies)):
        a, b = dependencies[i]
        c = outcomes[i]

        if x == c:
            x_i = i
            affects[a].remove(x)
            affects[b].remove(x)
            affects[a].append(y)
            affects[b].append(y)

        if y == c:
            y_i = i
            affects[a].remove(y)
            affects[b].remove(y)
            affects[a].append(x)
            affects[b].append(x)
    
    outcomes[x_i], outcomes[y_i] = y, x



seen_elems, all_swaps = set(), list()
outputs, carries, prev_carry = ["z00"], ["skt"], "skt"

for i in range(1, 45):
    x, y, z = (char + ["0", ''][i >= 10] + str(i) for char in ['x', 'y', 'z'])
    
    inf_x = get_influences_below_i(x, i)
    inf_y = get_influences_below_i(y, i)

    poss_cz = (inf_x | inf_y) - seen_elems # all the elements that depend solely on x00, x01, y00 and y01
    seen_elems = seen_elems | inf_x | inf_y
    
    c_poss, z_poss = find_carry_and_z(x, y, carries[-1], poss_cz)

    if z in z_poss and len(c_poss) >= 1: # great! it is correct
        carries.append(c_poss[-1])
        outputs.append(z)

    else: # not great, we need to try some swaps
        solved = False
        poss_cz = list(poss_cz)

        for i in range(len(poss_cz)):
            cz1 = poss_cz[i]
            for j in range(i+1, len(poss_cz)):
                cz2 = poss_cz[j]

                swap(cz1, cz2)

                c_poss, z_poss = find_carry_and_z(x, y, carries[-1], poss_cz)
                if z in z_poss and len(c_poss) >= 1:
                    carries.append(c_poss[-1])
                    outputs.append(z)
                    all_swaps.append(cz1)
                    all_swaps.append(cz2)
                    solved = True
                
                swap(cz1, cz2) # swap them back
                
            
            if solved:
                break




In [5]:
print(all_swaps)

print(",".join(sorted(all_swaps)))

['z06', 'fkp', 'ngr', 'z11', 'mfm', 'z31', 'bpt', 'krj']
bpt,fkp,krj,mfm,ngr,z06,z11,z31
